# Verschiedene Kurz-Analysen Gemeinderat
- Vorstösse
    - Wer wie viel und mit wem
- Fluktuationen im Parlament

In [247]:
import requests
import pandas as pd
import time 
import datetime
from datetime import timedelta
import glob
import re

### Vorstösse
Wer macht wie viel und mit wem zusammen

In [248]:
#Lese den Namen des letztmals aktualisierten Files ein.
filename = glob.glob("gemeinderat_stand_*.csv")[0]
df = pd.read_csv(filename)
df['names1'] = df['vorname_1'] + " " + df['name_1']
df['names2'] = df['vorname_2'] + " " + df['name_2']
df.head()

,GeschaeftId,Geschaeftsjahr,Geschaeftsnummer,Geschaeftstitel,Geschaeftsart,gruppe,id_1,name_1,vorname_1,partei_1,wahlkreis_1,id_2,name_2,vorname_2,partei_2,wahlkreis_2,names1,names2
0,675db553-31a1-4b9d-b791-0c8a6ecb6f69,2020,47,Flexibilisierung des Raumangebots und dessen M...,Postulat,NaN,6786f4ed-9d22-420e-a909-8d732dda6b5c,Zygmont,Dominique,FDP,NaN,54c0ca10-69df-4f6d-add5-f10d52aaebcc,Bourgeois,Yasmine,FDP,NaN,Dominique Zygmont,Yasmine Bourgeois
1,a2b58bba-2fa0-4f1b-b481-1356cd159472,2020,46,Schaffung einer unabhängigen Ombudsstelle im B...,Postulat,Grüne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0a490335-618d-45dc-8038-59377944d264,2020,45,Unterstützung der Sozialpartner betreffend Aus...,Postulat,Grüne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,58f00ca4-f4f8-42e6-a5ba-88f36fa0ad97,2020,44,Massgebliche und generelle Erhöhung der Qualit...,Motion,Grüne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e0151272-b8d7-4cae-a2a3-58952cd5ad3f,2020,43,Deutliche Senkung der Elternbeiträge für die f...,Motion,Grüne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Wer macht wie viele Vorstösse

In [249]:
#data = datensatz
#year = minimum jahr
#geschaeftsnr = für legislatur analysen, erstes geschäfte der legislatur, ansonsten 1 wählen
def get_vorstoesse_count(data,year,geschaeftsnr):
    data = data[~((data['Geschaeftsjahr']==year)&(data['Geschaeftsnummer']<geschaeftsnr))]
    data = data[data['Geschaeftsjahr']>=year]
    erst = data[['names1','id_1']].groupby('names1').count().reset_index()
    erst['id_2'] = data[['names2','GeschaeftId']].groupby('names2').count().reset_index().GeschaeftId
    erst['total'] = erst['id_1']+erst['id_2']
    erst = erst.sort_values('total', ascending=False)
    erst.columns = ['Name', 'erst', 'zweit', 'total']
    return erst


In [250]:
rangliste = get_vorstoesse_count(df,2018,1)
rangliste.head(20)

,Name,erst,zweit,total
8,Balz Bürgisser,28,8,36
27,Elisabeth Schoch,19,6,25
118,Yasmine Bourgeois,13,11,24
63,Martina Zürcher,3,18,21
57,Markus Knauss,16,5,21
116,Walter Angst,3,17,20
107,Stephan Iten,16,1,17
49,Marcel Savarioud,8,9,17
61,Martin Götzl,13,3,16
19,David Garcia Nuñez,9,7,16


#### Wer macht mit wem Vorstösse

In [260]:
#data = datensatz
#year = minimum jahr
#name = Vorname Nachname | ohne Partei
#geschaeftsnr = für legislatur analysen, erstes geschäfte der legislatur, ansonsten 1 wählen
def get_partners(data, name, year, geschaeftsnr):
    polid = str(list(set(data['id_1'][data['names1']==name]))[0])
    polparty = str(list(set(data['partei_1'][data['names1']==name]))[0])
    
    subset = data[(data['id_1']==polid) & (data['Geschaeftsjahr']>=year) &~((data['Geschaeftsjahr']==year)&(data['Geschaeftsnummer']<geschaeftsnr)) | (data['id_2']==polid) & (data['Geschaeftsjahr']>=year) & ~((data['Geschaeftsjahr']==year)&(data['Geschaeftsnummer']<geschaeftsnr))]
    print(name + ' (' + polparty + ')' + ' hat ' + 'seit ' + str(year) +' (G-Nr: '+str(geschaeftsnr)+') '+ str(len(subset)) + ' Vorstösse eingereicht.')
        
    print('\n******\n')
    print('Das waren die Vorstossarten\n')
    art = pd.DataFrame(subset.Geschaeftsart.values.tolist())
    print(art[0].value_counts())
    print('\nUnd in Prozent:')
    print(art[0].value_counts('normalize'))
    
    print('\n*****\n')
    print('Zusammen mit:\n')    

    partnerlist = pd.DataFrame(subset.names1.values.tolist() + subset.names2.values.tolist())
    partnerlist = partnerlist[(partnerlist[0]!='NaN NaN') & (partnerlist[0]!=name)]
    
    print(partnerlist[0].value_counts())
    print('\n****\n')
    print('So oft waren andere Parteien involviert\n')
    partnerparty = pd.DataFrame(subset.partei_1.values.tolist() + subset.partei_2.values.tolist())
    partnerparty = partnerparty[(partnerparty[0]!='NaN') & (partnerparty[0]!=polparty)]    
    print(partnerparty[0].value_counts())
    print('\nUnd in Prozent:')
    print(partnerparty[0].value_counts('normalize'))



In [261]:
#get_partners('Vorname Name', Jahr) | Jahr == 0 ganze Zeitraum, z.B. 2018 == seit 2018
get_partners(df,'Ezgi Akyol', 2018, 1)

Ezgi Akyol (AL) hat seit 2018 (G-Nr: 1) 13 Vorstösse eingereicht.

******

Das waren die Vorstossarten

Schriftliche Anfrage    6
Postulat                5
Globalbudgetantrag      1
Interpellation          1
Name: 0, dtype: int64

Und in Prozent:
Schriftliche Anfrage    0.461538
Postulat                0.384615
Globalbudgetantrag      0.076923
Interpellation          0.076923
Name: 0, dtype: float64

*****

Zusammen mit:

David Garcia Nuñez    6
Luca Maggi            4
Natalie Eberle        1
Nadia Huberson        1
Anjushka Früh         1
Name: 0, dtype: int64

****

So oft waren andere Parteien involviert

Grüne    4
SP       2
Name: 0, dtype: int64

Und in Prozent:
Grüne    0.666667
SP       0.333333
Name: 0, dtype: float64


### Mitglieder
Fluktuationen im Rat

In [262]:
#Lese den Namen des letztmals aktualisierten Files ein.
filename = glob.glob("alle_raete_*.csv")[0]
df = pd.read_csv(filename)
df.head()

,Id,Name,Vorname,Titel,Partei,Wahlkreis,WahlkreisOrderBy,vornachname
0,7f7b4dc0-5867-443e-a4d9-15b42f9bdf0a,Abele,Martin,NaN,Grüne,NaN,0,Martin Abele
1,891de33c-c98b-4325-b735-b1a078dbc368,Ackermann,Ruth,NaN,CVP,NaN,0,Ruth Ackermann
2,5326241e-9551-48bd-9322-1363cc38c5a3,Aebi,Hermann,NaN,Parteilos,NaN,0,Hermann Aebi
3,29fe0193-39b8-4945-9e4a-e1bd7a8b177e,Aeschbach,Christian,NaN,FDP,NaN,0,Christian Aeschbach
4,afb2e0bc-b941-4fa4-8936-9b4978fc1ee1,Akyol,Ezgi,NaN,AL,4 und 5,102,Ezgi Akyol


In [263]:
#Lade Details für alle Ratsmitglieder herunter
Ids = df.Id.to_list()
df_vonbis = pd.DataFrame()
for Id in Ids:
    url_mitglied = 'https://www.gemeinderat-zuerich.ch/api/Mitglieder/details?mid='+ Id
    response = requests.get(url_mitglied)
    mitglied = response.json()
    data = pd.DataFrame(mitglied['GruppenMitgliedschaften'])
    data['Id'] = Id
    df_vonbis = pd.concat([df_vonbis, data])
df_vonbis.head()

,Von,Bis,Name,Id
0,2006-05-03T00:00:00,2015-10-01T00:00:00,Gemeinderat,7f7b4dc0-5867-443e-a4d9-15b42f9bdf0a
0,2013-01-10T00:00:00,2015-05-27T00:00:00,Gemeinderat,891de33c-c98b-4325-b735-b1a078dbc368
1,2006-05-03T00:00:00,2010-05-18T00:00:00,Gemeinderat,891de33c-c98b-4325-b735-b1a078dbc368
0,1981-06-03T00:00:00,2002-02-27T00:00:00,Gemeinderat,5326241e-9551-48bd-9322-1363cc38c5a3
0,2004-07-08T00:00:00,2010-08-22T00:00:00,Gemeinderat,29fe0193-39b8-4945-9e4a-e1bd7a8b177e


In [264]:
#Merge der Datensätze und säubere Zeitformat
df = pd.merge(df, df_vonbis[['Von', 'Bis', 'Id']])
df['Von'] = df['Von'].str.replace('T00:00:00', '').str.strip()
df['Bis'] = df['Bis'].str.replace('T00:00:00', '').str.strip()
#Lade das heutige Datum ein (für spätere Berechnungen)
today = datetime.datetime.today().strftime('%Y-%m-%d')
df['Bis'][df['Bis'].isna()] = today
df['Von'] = pd.to_datetime(df['Von'], format='%Y-%m-%d')
df['Bis'] = pd.to_datetime(df['Bis'], format='%Y-%m-%d')
df['vornachname'] = df['Vorname'] + ' ' + df['Name']
df.head()

/Users/patricesiegrist/.pyenv/versions/3.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Id,Name,Vorname,Titel,Partei,Wahlkreis,WahlkreisOrderBy,vornachname,Von,Bis
0,7f7b4dc0-5867-443e-a4d9-15b42f9bdf0a,Abele,Martin,NaN,Grüne,NaN,0,Martin Abele,2006-05-03,2015-10-01
1,891de33c-c98b-4325-b735-b1a078dbc368,Ackermann,Ruth,NaN,CVP,NaN,0,Ruth Ackermann,2013-01-10,2015-05-27
2,891de33c-c98b-4325-b735-b1a078dbc368,Ackermann,Ruth,NaN,CVP,NaN,0,Ruth Ackermann,2006-05-03,2010-05-18
3,5326241e-9551-48bd-9322-1363cc38c5a3,Aebi,Hermann,NaN,Parteilos,NaN,0,Hermann Aebi,1981-06-03,2002-02-27
4,29fe0193-39b8-4945-9e4a-e1bd7a8b177e,Aeschbach,Christian,NaN,FDP,NaN,0,Christian Aeschbach,2004-07-08,2010-08-22


In [265]:
timediffs = (df['Bis'] - df['Von'])
df['days'] = [timediff.days for timediff in timediffs]
df['years'] = df['days']/365
df.head()

,Id,Name,Vorname,Titel,Partei,Wahlkreis,WahlkreisOrderBy,vornachname,Von,Bis,days,years
0,7f7b4dc0-5867-443e-a4d9-15b42f9bdf0a,Abele,Martin,NaN,Grüne,NaN,0,Martin Abele,2006-05-03,2015-10-01,3438,9.419178
1,891de33c-c98b-4325-b735-b1a078dbc368,Ackermann,Ruth,NaN,CVP,NaN,0,Ruth Ackermann,2013-01-10,2015-05-27,867,2.375342
2,891de33c-c98b-4325-b735-b1a078dbc368,Ackermann,Ruth,NaN,CVP,NaN,0,Ruth Ackermann,2006-05-03,2010-05-18,1476,4.043836
3,5326241e-9551-48bd-9322-1363cc38c5a3,Aebi,Hermann,NaN,Parteilos,NaN,0,Hermann Aebi,1981-06-03,2002-02-27,7574,20.750685
4,29fe0193-39b8-4945-9e4a-e1bd7a8b177e,Aeschbach,Christian,NaN,FDP,NaN,0,Christian Aeschbach,2004-07-08,2010-08-22,2236,6.126027


In [266]:
## Laufende Legislatur
#erster Tag der Legislatur: 2018-05-16
firstday = '2018-05-16'
neue = df[(df['Von']>=firstday) & (df['Bis']==today)]
print(len(neugewaehlte), 'sind neu in dieser Legilsatur dabei mit dabei')
print(len(df[(df['Von']>=firstday) & (df['Bis']<today)]), 'Rücktritte gab es von Neugewählten.')
print(len(df[(df['Bis']>=firstday) & (df['Bis']<today)]), 'Rücktritte gab es in der laufenden Legislatur.')
ruecktritte = df[(df['Bis']>=firstday) & (df['Bis']<today)]
print(round(df[(df['Bis']>firstday) & (df['Bis']<today)].years.median(),1), 'Jahre waren die zurückgetretenen im Schnitt am Stück dabei')
print((round(df[(df['Bis']==today)].years.median(),1)), 'Jahre sind die aktuellen Räte im Schnitt dabei.')
print('Durchschnittliche Amtsdauer (Median) der Parteien\n\n',df[(df['Bis']==today)].groupby('Partei').years.median(), '\n\n')
print('Aktuell am längsten dabei', df[['vornachname', 'Partei','years']][df['Bis']==today].sort_values('years', ascending=False).head(10))

35 sind neu in dieser Legilsatur dabei mit dabei
1 Rücktritte gab es von Neugewählten.
14 Rücktritte gab es in der laufenden Legislatur.
4.7 Jahre waren die zurückgetretenen im Schnitt am Stück dabei
5.8 Jahre sind die aktuellen Räte im Schnitt dabei.
Durchschnittliche Amtsdauer (Median) der Parteien

 Partei
AL       4.678082
EVP      1.723288
FDP      5.750685
GLP      5.430137
Grüne    5.750685
SP       5.750685
SVP      5.750685
Name: years, dtype: float64 


Aktuell am längsten dabei               vornachname Partei      years
283         Joe A. Manser     SP  30.797260
209  Bernhard im Oberdorf    SVP  23.279452
235         Markus Knauss  Grüne  21.841096
387      Marcel Savarioud     SP  21.841096
256         Albert Leiser    FDP  20.460274
363           Mark Richli     SP  18.082192
15           Walter Angst     AL  17.832877
33        Roger Bartholdi    SVP  17.832877
225     Hans Jörg Käppeli     SP  17.082192
421     Christine Seidler     SP  16.353425


In [267]:
## vorherige Legislatur
firstday = '2014-05-07'
lastday = '2018-05-15'
print (len(df[(df['Bis']>=firstday) & (df['Bis']<lastday)]), 'neu eingetretene Ratsmitglieder traten während der Legislatur 14/18 ab.')
print(round(df[(df['Bis']>=firstday) & (df['Bis']<lastday)].years.median(),1), 'Jahre waren sie im Schnitt am Stück dabei.')
print (len(df[(df['Von']>=firstday) & (df['Bis']<lastday)]), 'neu eingetretene Ratsmitglieder traten während der Legislatur 14/18 ab.')
print(round(df[(df['Bis']>=firstday) & (df['Bis']<lastday)].years.median(),1), 'Jahre waren die zurückgetretenen in der Legislatur 14/18 im Schnitt am Stück dabei')

44 neu eingetretene Ratsmitglieder traten während der Legislatur 14/18 ab.
5.1 Jahre waren sie im Schnitt am Stück dabei.
9 neu eingetretene Ratsmitglieder traten während der Legislatur 14/18 ab.
5.1 Jahre waren die zurückgetretenen in der Legislatur 14/18 im Schnitt am Stück dabei
